# [Combinatorics](https://en.wikipedia.org/wiki/Combinatorics)

**ENGSCI233: Computational Techniques and Computer Systems** 

*Department of Engineering Science, University of Auckland*

# What's in this Notebook?

Combinatorics is a broad class of mathematics concerned with counting, arrangement and selection from finite, discrete sets of items. Numerical applications of combinatorics include:

- Arranging a set of items in a particular order according to some criteria, e.g., **sorting** a list of integers from largest to smallest.
- Looking within a network structure for a particular item, i.e., **searching**.
- Finding the number of subsets within a larger set that satisfy some criteria, e.g., the number of ways to get from A to F through a network of nodes. This is **enumeration**.

In this notebook, we'll look at examples of all three: searching, sorting and enumeration. You need to know:

- The difference between breadth and depth-first search - one prioritises looking across a network, the other down network branches - and how they are implemented.
- Despite sort algorithms achieving the same end, why different methods might be preferred in different circumstances. Some require less memory, others will work as they receive new data, and some have excellent scaling properties.
- How Dijkstra's algorithm works, and how to implement it in a computer program.

In [ ]:
# imports and environment: this cell must be executed before any other in the notebook
%matplotlib inline
from combinatorics233 import*

First, we'll need some definitions:

*** Finite, discrete sets ***

Consider the finite, discrete set, $S$, which is an unordered collection of some countable number of objects, e.g.,

$$ S=\{A,\,B,\,C,\,D,\,E,\,F\},\quad\quad S=\{1,\,2,\,3,\,...,\,n\}, \quad\quad S=\{-3.2,\,5.6,\,0.0,\,\text{None},\,[3, 2]\}.$$

Members of the set might be **repeated**, although this can lead to difficulty for search and sort algorithms.

$$ S=\{1,\,2,\,3,\,2\},\quad\quad S=\{\text{'alpha'},\,\text{'beta'},\,\text{'oranges'},\,\text{'gamma'},\,\text{'alpha'}\}$$

*** Combinations ***

Combinations are unordered subsets, $S_c$, derived from a larger set, $S$. We might subject combinations to some **rule**, e.g.,  combinations of **three items** from the set of four items

$$S=\{A,\,B,\,C,\,D\},\quad\quad S_c=\left\{\{A,\,B,\,C\},\,\{A,\,B,\,D\},\,\{A,\,C,\,D\},\,\{B,\,C,\,D\}\right\}$$

*** Permutations ***

Permutations are combinations for which the item **ordering** is a distinguishing property, i.e., the permutation $\{A,\,B,\,C\}$ is **different** to the permutation $\{A,\,C,\,B\}$. In constructing the set of permutations, $S_p$, we may also apply rules, e.g., subsets of three items from a larger set of four.

$$\rightarrow \text{different combinations}\rightarrow$$

$$S_p=\begin{Bmatrix}
\{A,\,B,\,C\},\,\{A,\,B,\,D\},\,\{A,\,C,\,D\},\,\{B,\,C,\,D\} \\
\{A,\,C,\,B\},\,\{A,\,D,\,B\},\,\{A,\,D,\,C\},\,\{B,\,D,\,C\} \\
\{B,\,A,\,C\},\,\{B,\,A,\,D\},\,\{C,\,A,\,D\},\,\{C,\,B,\,D\} \\
\{B,\,C,\,A\},\,\{B,\,D,\,A\},\,\{C,\,D,\,A\},\,\{C,\,D,\,B\} \\
\{C,\,A,\,B\},\,\{D,\,A,\,B\},\,\{D,\,A,\,C\},\,\{D,\,B,\,C\} \\
\{C,\,B,\,A\},\,\{D,\,B,\,A\},\,\{D,\,C,\,A\},\,\{D,\,C,\,B\} \end{Bmatrix},\quad\downarrow\,\,\text{different orderings}$$


***Enumeration***

This refers **either** to the ordering or counting of elements in a set. For instance, enumeration of the set of permutations above may return either the ordered set from most to least alphabetical ($\{A,\,B,\,C\}$ to $\{D,\,C,\,B\}$), or simply the size of the set, 24. When enumeration refers to putting set members **in order**, then we must consider the problem of **sorting**. 


## 1 Searching

<mark>***Algorithms to find particular entries in a network.***</mark>

We shall start by imagining that a **tree** network must be searched for the node containing a specific value.

A tree is a special kind of network that starts from a single **head node**. The head node is referred to as a **mother** and the nodes that it points to are called its **daughters** (or, alternatively, parent and child nodes). Each daughter node can have its own daughters. In this way, the tree network is conceptually similar to a genealogical family tree. 

See the example below for a visualization of a tree.

In [ ]:
# use nested tuples to define tree structure
tree_tuple = ('A',(('B',(('D',None),('E',None),('F',None))),('C',(('G',None),('H',None)))))

# define the tree
tree=Tree()
tree.build(tree_tuple)

# assign values tonodes
tree_vals = {'A':2,'B':-1,'D':3,'E':0,'F':-2,'C':1,'G':-3,'H':4}
tree.assign_values(tree_vals)

# plot the tree
tree.show()

***Comment the call to ```assign_values```. What happens and why?***

> <mark>*~ your answer here ~*</mark>

***How many "generations" are in this tree?***

> <mark>*~ your answer here ~*</mark>

***In what order would you go "searching" the nodes of this tree?***

> <mark>*~ your answer here ~*</mark>


### 1.1 Depth-first vs. breadth-first searching

Searching a tree involves finding a particular node holding a particular value. For instance, consider finding the node with value `1` in the tree above (clearly, we can see this is node `C`). 

There are two search strategies, both begin at the head node

- **Breadth-first** searching checks all nodes in a generation before moving on to the next.
- **Depth-first** searching explores down "family lines" in the tree. 

The visualisation below compares depth-first vs. breadth-first searches. 

In [ ]:
# this cell creates a visualisation of BREADTH-first and DEPTH-first searching
interact(interactive_search, search={'depth':0, 'breadth':1}, check=(1,8,1), tree = fixed(tree));

***- - - - CLASS CODING EXERCISE - - - -***

In [ ]:
# PART ONE
# --------
# UNCOMMENT and TEST each command below
# WRITE a comment to describe what each does

# initial variable values
cnt = 0
search_value = -4
ndA = tree.head

# **uncomment and test**
# ndA.value == search_value
# ndB = ndA.get('B')
# cnt = cnt + 1

In [ ]:
# OPTIONAL CHALLENGE
# ------------------
# TEST the command below

# abs(ndA.value - search_value) < 1.e-16
# nd = ndA.arcs_out[0].to_node
# cnt += 1

In [ ]:
# PART TWO
# --------
# HARD CODE commands to search, breadth-first, the tree network above.
# (hard-code = don't worry about making your code general to ANY tree)
#
# SEARCH_VALUE could change, so you should visit every node in the tree.

# initialisation
# --------------
cnt = 0              # a counter for how many operations have taken place
search_value = -4    # value to look for

# all searches start with the head node
ndA = tree.head
___    # increment counter

# check if found
if ndA.value == search_value:
    print('value found at node {} in {:d} operations'.format(node.name, cnt))
___    # increment counter

# get another node
ndB = ndA.get('B')
___    # increment counter
    
# copy-paste to build up your search algorithm, getting nodes, checking values
___


### 1.2 Depth-first search algorithm

A depth-first search can be implemented **recursively**, by searching a node's daughters, and its daughters' daughters and so on:
```
1. Initialise search value, X.
2. Call recursive function CHECK_NODE on head node.

CHECK_NODE(ND, X)
    if ND.value is X then
        return ND
    else if ND.daughters is None then
        return None
    else
        for daughter in ND.daughters do
            CHECK_NODE(daughter, X)
```
This algorithm is implemented in Python below.

In [ ]:
# recursive implementation of depth-first search
def depth_search(tree, value, verbose=False):
    ''' Initialise the search of TREE for VALUE.
    '''
    # display optional screen info about the search 
    if verbose:
        print('searching tree for value {}'.format(value))
        print('beginning at top node {}'.format(tree.head.name))
        
    # call recursive function on uppermost node
    check_node(node=tree.head, value=value, verbose=verbose)
    
def check_node(node, value, verbose):
    ''' Check NODE for VALUE. If not found, call CHECK_NODE on NODE's daughters.
    '''
    # check node for value
    if node.value == value:     # <--- living dangerously here, see next module on Computer Zero for more
        if verbose: print('{} FOUND at {}'.format(value, node.name))
        return node
    
    # VALUE not found, call CHECK_NODE recursively on daughters
    if verbose:
        print('checking node {}, with daughters: {}'.format(node.name, [arc.to_node.name for arc in node.arcs_out]))
        
    for arc in node.arcs_out:
        node = check_node(node=arc.to_node, value=value, verbose=verbose)
        # return only FOUND result
        if node is not None:
            return node
    
    # node not found in daughters, return None
    return None

# call the depth-first search
depth_search(tree=tree, value=-3, verbose=True)

### 1.3 Breadth-first search algorithm

This algorithm uses the **linked list** structure discussed in the previous module implemented as a **queue**. Nodes that are to eventually be searched are appended to the **end** of the queue, while the next node to be checked is popped from the **front**. The algorithm proceeds as follows:

```
1. Initialise search value, X.
2. Initialise the queue with the head node the only member.
3. Loop over:
  i. Pop the next node from the queue.
  ii. If value is found, end search.
  iii. If not, append the current node's daughters to the end of the queue.
```

**For now, let's imagine what the queue should look like as it is slowly built up.**

***- - - - CLASS CODING EXERCISE - - - -***

In [ ]:
# PART ONE
# --------
# What do the commands below do?

# create a linked list and add the head node
ll = LinkedList()
print("add first node")
ll.append(ndA)
print(ll)

# pop a node off the front
print("pop a node")
nd = ll.pop(0)
print(ll)

# add in the daughter nodes
print("add its daughters")
for arc in nd.arcs_out:
    ll.append(arc.to_node)
print(ll)


In [ ]:
# PART TWO
# --------
# WRITE commands below (use copy-paste) to keep adding and popping from the queue
# **your code here**


In [ ]:
# OPTIONAL CHALLENGE
# ------------------
# ADD a step to check for the search_value
# REPLACE duplicated commands inside a loop
# **your code here**


### 1.4 Depth-first search 2: the stack

An alternative implementation of a depth-first search uses the linked list as a **stack**. In a stack, nodes are still appended to the **end** of the linked list. However, instead of being popped from the front, they are now popped from the **end** (last in, first out). 

## 2 Sorting

<mark>***Algorithms to impose order (e.g., smallest to largest) on a disordered array.***</mark>

Sorting methods are widely used in numerical computation. Four related sorting tasks are identified.

- Rearrange an array of items into ascending or descending order by value (**descending, here**). 

$$\begin{matrix} \text{Position:}&\quad 0 \quad 1 \quad 2 \quad 3 \quad 4 \quad 5 \quad 6 \\ \text{Unsorted:}&\quad 5 \quad 2 \quad 1 \quad 4 \quad 6 \quad 7 \quad 3 \\ \text{Sorted:}&\quad 7 \quad 6 \quad 5 \quad 4 \quad 3 \quad 2 \quad 1 \end{matrix}$$

- Rearrange an array of values into ascending or descending order and **also rearrange** additional arrays to **preserve correspondence** between elements (e.g., names and marks). This is best done using an **index table**.

- Generate an **Index Table** of pointer values to the elements of an array in the sorted list (this is a specific case of the above where the "additional array" is just an array of indices). Note the correspondence of over/under pairs in *Position-Unsorted* and *Sorted-Index* below.

$$\begin{matrix} \text{Position:}&\quad 0 \quad 1 \quad 2 \quad 3 \quad 4 \quad 5 \quad 6 \\ \text{Unsorted:}&\quad 5 \quad 2 \quad 1 \quad 4 \quad 6 \quad 7 \quad 3 \\ \\ \text{Sorted:}&\quad 7 \quad 6 \quad 5 \quad 4 \quad 3 \quad 2 \quad 1  \\ \text{Index:}&\quad 5 \quad 4 \quad 0 \quad 3 \quad 6 \quad 1 \quad 2\end{matrix}$$

- Generate a **Rank Table**, which gives the position of each element in the sorted sequence:

$$\begin{matrix} \text{Position:}&\quad 0 \quad 1 \quad 2 \quad 3 \quad 4 \quad 5 \quad 6 \\ \text{Unsorted:}&\quad 5 \quad 2 \quad 1 \quad 4 \quad 6 \quad 7 \quad 3 \\ \text{Sorted:}&\quad 7 \quad 6 \quad 5 \quad 4 \quad 3 \quad 2 \quad 1  \\ \text{Rank:}&\quad 2 \quad 5 \quad 6 \quad 3 \quad 1 \quad 0 \quad 4 \end{matrix}$$

We shall consider two sorting algorithms (there are many others): **insertion sort** and **heap sort**.

***Explain how the expression `Sorted[i] = Unsorted[Index[i]]` is accurate in regards to index tables.***

> <mark>*~ your answer here ~*</mark>

***Explain how the expression `Unsorted[i] = Sorted[Rank[i]]` is accurate in regards to rank tables.***

> <mark>*~ your answer here ~*</mark>

### 2.1 Comparison

A key step in all sorting algorithms is **comparison** of two values: this is the basis for determining which item should precede the other. 

If the values are integer or floats, the comparison is undertaken straightforwardly using the `<` or `>` operators.

If instead the items are strings, e.g., `'cat'` and `'dog'`... we can still use the  `<` or `>` operators. 

***Execute the code snippet below and explain the results 1-6.***

In [ ]:
# 1. compare two numbers
a,b = [3.2, 5.8]
print('a = {}, b = {}, a<b is {}'.format(a,b,a<b))

# 2. compare two strings
a,b = ['a', 'b']
print('a = \'{}\', b = \'{}\', a<b is {}'.format(a,b,a<b))

# 3. compare the ascii code of two strings
a,b = [ord('a'), ord('b')]
print('a = {}, b = {}, a<b is {}'.format(a,b,a<b))

# 4. compare two longer strings
a,b = ['coffee','tea']
print('a = \'{}\', b = \'{}\', a<b is {}'.format(a,b,a<b))

# 5. compare two longer strings sharing the same first letter
a,b = ['toffee','tea']
print('a = \'{}\', b = \'{}\', a<b is {}'.format(a,b,a<b))

# 6. compare two longer strings sharing the same first letter but different case
a,b = ['Toffee','tea']
print('a = \'{}\', b = \'{}\', a<b is {}'.format(a,b,a<b))

[***How does string comparison work?***](http://stackoverflow.com/questions/4806911/string-comparison-technique-used-by-python)

> <mark>*~ your answer here ~*</mark>


### 2.2 Insertion sort

The pseudocode for insertion sort is presented below, sorting an input array, $A$, containing a sequence of $n$ numbers into ascending order. $A$ is sorted **in place**, which saves on memory but destroys the original order.

```
1    INSERTION-SORT(A)
2    for j = 1 to n-1 do
3       key = A[j]
4       i = j-1
5       while i>-1 do
6           if not (A[i] > key)
7               break
8           A[i+1] = A[i]
9           i = i-1
10      A[i+1] = key
```

Insertion sort is relatively easy to implement but is computationally expensive. It should only be used for small datasets. 

The cell below visualises the steps of insertion sort, which is implemented in `combinatorics233.py`. It is straightforward to modify this code to produce an **index table**.

In [ ]:
# run this cell for a visualisation of INSERTION sort
           
def interact_sort(step=0):
    # array for sorting (you can change this, but then you may need more steps below)
    A = [5, 3, 7, 1, 2]
    
    # call insertion sort method
    insertion_sort(A, step)
        
interact(interact_sort, step = (0,24,1));

***In the pseudocode for insertion sort above, to which lines do the green, red, blue and grey operations refer?*** 

> <mark>*~ your answer here ~*</mark>

***- - - - CLASS CODING EXERCISE - - - -***

In [ ]:
# PART ONE
# --------
# Move the SLIDER above to STEP 8
# partially sorted array A is
A = [3,5,7,1,2]
# the key is in position 4
key = A[3]

# write a while loop with a DECREASING array index:
# - that BEGINS at the first entry TO THE LEFT of 'key'
# - HALT the while loop at the BEGINNING of the array
# - PRINT the array value
    
# HINT: if you accidentally start an infinite loop, double tap the '0' key to restart Python

In [ ]:
# PART TWO
# --------
# MODIFY your while loop above so that
# - it takes a GENERAL beginning index key, j
# - uses a FOR loop to iterate j over the length of the array.
# - PRINT the current key AND the array value
A = [5, 3, 7, 1, 2]
        

In [ ]:
# OPTIONAL CHALLENGE
# ------------------
# modify your loops above to include ARRAY SWAPS and LOOP BREAKS according to the pseudocode above.

### 2.3 Properties of sort algorithms

There are a large number of possible sort algorithms: [bubble](https://en.wikipedia.org/wiki/Bubble_sort), [insertion](https://en.wikipedia.org/wiki/Insertion_sort), [Shell](https://en.wikipedia.org/wiki/Shellsort), [mergesort](https://en.wikipedia.org/wiki/Merge_sort), [quicksort](https://en.wikipedia.org/wiki/Quicksort), [timsort](https://en.wikipedia.org/wiki/Timsort), [heapsort](https://en.wikipedia.org/wiki/Heapsort), etc. Surely one of these is "the best", and we should use just ***that*** algorithm all the time?

In practice, different sort algorithms have unique properties that may suit them to a particular task better than others. ***This idea applies to algorithms in general and is an essential take-away from this course.***

For example, quicksort is **quite fast** for the average input, but can require a lot of memory. Shell's sort does not scale quite so well, but has **minimal memory requirements**. We might use one algorithm for a [microbit](https://microbit.org/) with limited memory, and another one on a supercomputer.

When choosing, one should consider the following properties of sort algorithms:

***2.3.1 Scaling***

Recall from the **Performance** module the concept of algorithm scaling, quantified in terms of Big O Notation, $\mathcal{O}()$. To measure the efficiency of a sorting algorithm, we count the number of operations to complete the sort in two situations: 

1. Worst case performance, where the data is initially ordered the opposite way to desired (e.g., initially in ascending order, we desire to sort in descending).

2. Average case performance, where the data is in a random order.

***2.3.2 Memory***

Algorithms that rely on **recursion**, such as quicksort, have the potential to grow quite rapidly in terms of required memory. 

***2.3.3 Stability***

When a sorting algorithm encounters two identical values in a list, the algorithm is **stable** if the original order of the two items is **preserved**.

***2.3.4 Online***

An algorithm can sort a list as it is received, i.e., the complete list is not required before sorting begins. Insertion sort is an online algorithm.

***Run the next cell and then answer the questions below.***

In [ ]:
# compare insertion and heap sort operations for increasing N
import numpy as np
from matplotlib import pyplot as plt
N = np.linspace(1,1e5, 101)
k = 0.3e4
heapN = k*N*np.log2(N)
insertionN = N**2

# plot the comparison
f,ax = plt.subplots(1,1)
ax.plot(N, insertionN, 'b-', label='insertion')
ax.plot(N, heapN, 'r-', label = 'heap')
ax.legend()
ax.set_xlabel('length of list')
ax.set_ylabel('number of operations');

***Which algorithm is more efficient for shorter length lists? For longer lengths?***

> <mark>*~ your answer here ~*</mark>

***How does the constant factor $k$  affect your answer above? (change it in the code and see)***

> <mark>*~ your answer here ~*</mark>

***How would you decide which sorting algorithm to use?***

> <mark>*~ your answer here ~*</mark>


## 3 Dijkstra's algorithm

<mark>***Finding the shortest path through a network.***</mark>

Given a network comprising a set of nodes linked by arcs, we are sometimes interested in finding the shortest route between a source node and a destination node. 

***Describe a situation in which it might be important to find the shortest path.***

> <mark>*~ your answer here ~*</mark>

[Dijkstra's **shortest path** algorithm](https://en.wikipedia.org/wiki/Dijkstra's_algorithm) achieves this by (1) finding the shortest distance to all other nodes in the network, (2) stopping when the shortest distance to the destination node is determined. 

The algorithm steps are:

**Initialisation**: Form two sets of nodes. Set 1 consists of *solved nodes*. Set 2, called the **unvisited set**, consists of *unsolved nodes*. Every node must be a member of only one of these sets at any given time. *Initially all the nodes are members of the unvisited set.* 

Assign each node a **distance**, $d$, that is a very large number (`node.distance = float("Inf")`) and, eventually, a predecessor node, $p$. The one exception is the **source node**, which has distance 0 and no predecessor.

**Iterations**:

1. From all nodes in the unvisited set, choose node $i$ with minimum distance $d(i)$. Node $i$ becomes a solved node now. In case of a tie, any one of the unsolved nodes becomes a solved node.

2. List the unsolved nodes which can be reached by following a single arc out of the new solved node $i$ identified in Step 1.

3. Find the total distance from the origin to each of the unsolved nodes $j$ listed in Step 2.
		
   Total distance =	Shortest distance from origin to solved node + Distance from solved node to adjacent unsolved node
   OR
   $\bar{d}(j) = d(i) + \text{weight of arc}(i,j)$
   
   If this total distance $\bar{d}(j)$ is **smaller** than the current distance $d(j)$, update the distance ($d(j)\leftarrow \bar{d}(j)$) and set the predecessor of node $j$ as node $i$.

**Stopping Criterion**: Repeat Steps 1, 2, and 3 until the **destination node** becomes a solved node.

**Discussion:**
- These shortest path algorithms will produce the shortest path to every node closer than the destination as a by-product. By letting the algorithm run its course until there are no more unsolved nodes (instead of using the stopping criterion), we can produce the shortest path from the origin to every possible destination in the network.
- With non-negative arc weights, Dijkstra’s Algorithm is guaranteed to produce the optimal solution.

***Run the cell below for a demonstration of Dijkstra's algorithm***

In [ ]:
dijkstra_example()